# Calculating the score for different estimates of battery charge distribution and comparing these scores

## Compute score

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import sys
import os
sys.path.append(os.path.abspath('../common/'))
import common_functions as cf
import datetime

### Get real values for prediction week

In [3]:
demand_task2_path = '../task2/Input/demand_train_set2.csv'
solar_power_task2_path = '../task2/Input/pv_train_set2.csv'
weather_task2_path = '../task2/Input/weather_train_set2.csv'
dp_task2 = cf.DataPreprocesser(demand_path=demand_task2_path, solar_path=solar_power_task2_path, weather_path=weather_task2_path)
dp_task2.load_df()
dp_task2.remove_nan()
dp_task2.interpolate_df()

In [10]:
first_day_prediction = datetime.datetime(2018,10,16) 
demand_and_solar_power = dp_task2.df.loc[(dp_task2.df.index >= first_day_prediction) & 
                                         (dp_task2.df.index < first_day_prediction+ datetime.timedelta(days=7)), :]
demand_and_solar_power

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,solar_location3,solar_location6,solar_location2,solar_location4,solar_location5,solar_location1,week,dow,hour,sp
datetime,,,,,,,,,,,,,,,,,,,,
2018-10-16 00:00:00,1.95,0.0,0.0,12.41,11.650,14.870,12.670,11.450,14.190,11.740,0.0,0.0,0.0,0.0,0.0,0.0,42,1,0,1.0
2018-10-16 00:30:00,1.87,0.0,0.0,12.38,11.810,14.850,12.650,11.610,14.190,11.720,0.0,0.0,0.0,0.0,0.0,0.0,42,1,0,2.0
2018-10-16 01:00:00,1.80,0.0,0.0,12.40,11.970,14.830,12.630,11.770,14.190,11.700,0.0,0.0,0.0,0.0,0.0,0.0,42,1,1,3.0
2018-10-16 01:30:00,1.76,0.0,0.0,12.63,11.930,14.825,12.600,11.750,14.155,11.645,0.0,0.0,0.0,0.0,0.0,0.0,42,1,1,4.0
2018-10-16 02:00:00,1.76,0.0,0.0,12.91,11.890,14.820,12.570,11.730,14.120,11.590,0.0,0.0,0.0,0.0,0.0,0.0,42,1,2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-22 21:30:00,2.37,0.0,0.0,3.30,7.985,12.885,8.845,6.710,12.530,8.720,0.0,0.0,0.0,0.0,0.0,0.0,43,0,21,44.0
2018-10-22 22:00:00,2.11,0.0,0.0,3.15,8.010,12.760,8.980,6.220,12.400,8.890,0.0,0.0,0.0,0.0,0.0,0.0,43,0,22,45.0
2018-10-22 22:30:00,1.90,0.0,0.0,3.32,7.705,12.690,8.615,5.925,12.360,8.695,0.0,0.0,0.0,0.0,0.0,0.0,43,0,22,46.0


### Score for BGBattery prediction task1

In [14]:
class ScoreComputer2:
    def __init__(self, B_path):
        B = pd.read_csv(B_path, parse_dates=['datetime'],index_col=['datetime'])
        B['week']=pd.Int64Index(B.index.isocalendar().week)
        B['dow']=B.index.dayofweek
        B['hour'] = B.index.hour
        B['sp'] = B.hour*2 + B.index.minute/30 + 1
        self.B = B
        
    def compute_r_peak(self,demand_and_solar_power,week,dow, B_pred=None):
        if B_pred is None:
            B = self.B
        else:
            B = B_pred
        demand_discharge = demand_and_solar_power.loc[(demand_and_solar_power['week']==week) & 
                                          (demand_and_solar_power['dow']==dow)&(demand_and_solar_power['sp']>=32)&
                                          (demand_and_solar_power['sp']<=42),'demand_MW']
        B_discharge = B.loc[(B['week']==week) & (B['dow']==dow)&(B['sp']>=32) & (B['sp']<=42),'charge_MW']
        old_peak = demand_discharge.max()
        if old_peak == 0:
            return 0
        new_peak = (B_discharge + demand_discharge).max()
        return 100*(old_peak-new_peak)/old_peak
    def compute_p_solar(self,demand_and_solar_power, week, dow, B_pred=None):
        if B_pred is None:
            B = self.B
        else:
            B = B_pred
        solar_power_for_charge = demand_and_solar_power.loc[(demand_and_solar_power['week']==week) & 
                                              (demand_and_solar_power['dow']==dow)&(demand_and_solar_power['sp']<=31),['pv_power_mw']]
        B_charge = B.loc[(B['week']==week) & (B['dow']==dow)&(B['sp']<=31),['charge_MW']]
        solar_power_for_charge = solar_power_for_charge.merge(B_charge, how='outer', left_index=True, right_index=True)
        battery_charge = B_charge['charge_MW'].sum()*0.5
        solar_power_for_charge['charge_from_solar_MW'] = solar_power_for_charge.apply(lambda x: min(x['pv_power_mw'], x['charge_MW']), axis=1)
        battery_charge_from_solar = solar_power_for_charge['charge_from_solar_MW'].sum()*0.5
        if battery_charge == 0:
            p_solar = 0
        else:
            p_solar = battery_charge_from_solar / battery_charge
        p_grid = 1-p_solar
        return p_solar, p_grid, battery_charge, solar_power_for_charge
    def compute_scores(self, demand_and_solar_power, first_day_pred, B_pred=None):
        if B_pred is None:
            B = self.B
        else:
            B = B_pred
        scores = pd.DataFrame(columns = ['r_peak', 'p_solar', 's'])
        idx = 0
        dow_index = []
        with tqdm(total=7, file=sys.stdout) as pbar:
            for i in range(7):
                the_date = first_day_pred+datetime.timedelta(days=i)
                dow = the_date.weekday()
                week = the_date.isocalendar()[1]
                r_peak = self.compute_r_peak(demand_and_solar_power, week, dow, B)
                p_solar, p_grid, battery_charge, solar_power_for_charge = self.compute_p_solar(demand_and_solar_power, week, dow, B)
                s = r_peak*(3*p_solar + p_grid)
                scores.loc[idx, :] = [r_peak, p_solar, s]
                idx += 1
                dow_index.append(dow)
                pbar.update()
        scores.index = dow_index
        self.scores = scores
        self.scores_mean = scores.mean()
        return scores, scores.mean()


In [15]:
BGBattery_path = 'Output/BGBattery_set1.csv'

In [16]:
BGBattery_sc = ScoreComputer2(BGBattery_path)
BGBattery_sc.compute_scores(demand_and_solar_power, first_day_prediction)

100%|██████████| 7/7 [00:00<00:00, 80.12it/s]


(      r_peak   p_solar          s
 1  32.499739  0.680876  76.756309
 2  32.167428  0.624915  72.371238
 3  34.204007  0.994471  102.23377
 4  32.553062     0.923  92.646012
 5   32.97789       1.0  98.933669
 6  30.672629  0.999941  92.014247
 0   29.89071  0.999869  89.664297,
 r_peak     32.137924
 p_solar     0.889010
 s          89.231363
 dtype: float64)